## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/Datasets/processed_balanced_final.zip"

/content/drive/MyDrive/Datasets/processed_balanced_final.zip


## Unzip the processed_balanced_final folder

## Now we will Take this dataset and train a Yolo classifier

In [ ]:
!mkdir -p /content/Datasets/processed_balanced_final

#Check to see the 1st image:

In [ ]:
# Unzip the file into the same directory
!unzip -o "/content/drive/MyDrive/Datasets/processed_balanced_final.zip" -d "/content/Datasets/processed_balanced_final"


Streaming output truncated to the last 5000 lines.
  inflating: /content/Datasets/processed_balanced_final/processed_balanced_final/val/labels/A_DJ-RT-20230921-29_031370.txt  
  inflating: /content/Datasets/processed_balanced_final/processed_balanced_final/val/labels/A_DJ-RT-20230921-29_031371.txt  
  inflating: /content/Datasets/processed_balanced_final/processed_balanced_final/val/labels/A_DJ-RT-20230921-29_031372.txt  
  inflating: /content/Datasets/processed_balanced_final/processed_balanced_final/val/labels/A_DJ-RT-20230921-29_031373.txt  
  inflating: /content/Datasets/processed_balanced_final/processed_balanced_final/val/labels/A_DJ-RT-20230921-29_031374.txt  
  inflating: /content/Datasets/processed_balanced_final/processed_balanced_final/val/labels/A_DJ-RT-20230921-29_031375.txt  
  inflating: /content/Datasets/processed_balanced_final/processed_balanced_final/val/labels/A_DJ-RT-20230921-29_031376.txt  
  inflating: /content/Datasets/processed_balanced_final/processed_balanced

Copy and paste folder to instance

Yolo Model

Based on your training output, I can see you're getting about 0.9 iterations per second on the Tesla T4. Let me calculate the expected training time:
Time Estimates for Your Current Training
With 1,561 iterations per epoch (49,939 images ÷ 32 batch size) at 0.9 it/s:

Per epoch: ~29 minutes (1,561 ÷ 0.9 ÷ 60)
70 epochs total: ~34 hours

This is much longer than expected! The issue is that you're not getting the full T4 speed. Here's why and how to fix it:
The Problem

RAM caching failed - You got the warning that 85.7GB RAM is needed but only 12.7GB available
Disk I/O bottleneck - Without caching, it's reading images from disk each time
Speed is 0.9 it/s instead of expected 8-10 it/s

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.1 MB/s eta 0:00:00


Next Attempt:

In [ ]:
# Clear memory and restart
import torch
torch.cuda.empty_cache()
!nvidia-smi  # Check memory is clear

Wed Sep 10 07:11:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

To show progress

Attempted n model

In [ ]:
import os
import torch
from pathlib import Path
from datetime import datetime
from ultralytics import YOLO

# Mount Google Drive first
from google.colab import drive
drive.mount('/content/drive')

# Setup paths
data_path = Path('/content/Datasets/processed_balanced_final/processed_balanced_final')
output_dir = Path('/content/drive/MyDrive/Datasets/Models')  # <-- SAVE DIRECTLY TO DRIVE
output_dir.mkdir(parents=True, exist_ok=True)

# GPU check
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu_name} with {gpu_mem:.1f} GB memory")

# Optimized settings
model_size = 'n'
epochs = 60
batch_size = 32

print("="*60)
print(f"Training YOLOv8{model_size} for {epochs} epochs")
print("Expected time: ~6-7 hours")
print("Expected mAP50: 0.60-0.64")
print("="*60)

# Create experiment directory inside Google Drive
exp_name = f"yolov8{model_size}_fast_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
exp_dir = output_dir / exp_name

# Initialize model
model = YOLO(f'yolov8{model_size}.pt')

# Train with checkpoint saving
results = model.train(
    data=str(data_path / 'dataset.yaml'),
    device=0,
    epochs=epochs,
    batch=batch_size,
    imgsz=640,
    patience=10,
    save=True,
    save_period=10,   # 🔑 Save a checkpoint every 10 epochs
    project=str(exp_dir),
    name='train',
    exist_ok=True,
    pretrained=True,
    optimizer='SGD',
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=2,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    plots=True,
    val=True,
    cache='ram',
    workers=4,
    close_mosaic=10,
    amp=True,
    # Weld-specific augmentations
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.3,
    degrees=15,
    translate=0.15,
    scale=0.3,
    shear=5,
    perspective=0.0002,
    flipud=0.5,
    fliplr=0.5,
    mosaic=0.5,
    mixup=0,
    copy_paste=0.0,
    auto_augment=None,
)
    # # WELD-SPECIFIC AUGMENTATIONS
    # hsv_h=0.0,  # No hue for grayscale
    # hsv_s=0.0,  # No saturation for grayscale
    # hsv_v=0.3,  # Brightness variance (X-ray exposure differences)
    # degrees=15,  # Welds can be at various angles
    # translate=0.15,  # Weld position varies
    # scale=0.3,  # Different weld sizes
    # shear=5,  # Some shear is OK for industrial
    # perspective=0.0002,  # Slight perspective from X-ray angle
    # flipud=0.5,  # Defects have no orientation preference
    # fliplr=0.5,  # Full horizontal flip
    # mosaic=0.9,  # HIGH mosaic - great for small defects
    # mixup=0,  # Some mixup helps with small defects
    # copy_paste=0.0,  # Too slow, skip it
print(f"\nTraining complete!")
print(f"Checkpoints and best weights are saved under: {exp_dir/'train/weights'}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU: Tesla T4 with 14.7 GB memory
Training YOLOv8n for 60 epochs
Expected time: ~6-7 hours
Expected mAP50: 0.60-0.64
Ultralytics 8.3.197 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=32, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Datasets/processed_balanced_final/processed_balanced_final/dataset.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=No

KeyboardInterrupt: 

S model if I ever wanna try it later.

In [ ]:
# YOLOv8s - OPTIMIZED FOR WELD X-RAY INSPECTION ON T4

import torch
from pathlib import Path
from datetime import datetime
from ultralytics import YOLO

# Clear GPU memory
torch.cuda.empty_cache()

# Setup
data_path = Path('/content/Datasets/processed_balanced_final/processed_balanced_final')
output_dir = Path('/content/models')
output_dir.mkdir(parents=True, exist_ok=True)

# Configuration for weld inspection
model_size = 's'  # Still worth it for tiny defects
epochs = 50  # Reduced - weld patterns converge faster than medical
batch_size = 48  # Can push higher for industrial images

print("="*60)
print(f"Training YOLOv8{model_size} for Weld X-ray Inspection")
print(f"Target defects: porosity, cracks, lack of fusion, inclusions")
print(f"Epochs: {epochs}, Batch: {batch_size}")
print("Expected time: ~7-8 hours")
print("Expected mAP50: 0.65-0.70")
print("="*60)

exp_name = f"yolov8{model_size}_weld_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
exp_dir = output_dir / exp_name

model = YOLO(f'yolov8{model_size}.pt')

results = model.train(
    data=str(data_path / 'dataset.yaml'),
    device=0,
    epochs=epochs,
    batch=batch_size,
    imgsz=640,
    patience=15,  # Weld defects converge faster
    save=True,
    save_period=-1,  # Only save best and last
    project=str(exp_dir),
    name='train',
    exist_ok=True,
    pretrained=True,
    optimizer='SGD',  # SGD is fine for industrial - faster than AdamW
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=2,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    plots=True,
    val=True,
    cache='ram',  # CRITICAL - massive speedup after first epoch
    workers=4,
    close_mosaic=10,
    amp=True,
    # WELD-SPECIFIC AUGMENTATIONS
    hsv_h=0.0,  # No hue for grayscale
    hsv_s=0.0,  # No saturation for grayscale
    hsv_v=0.3,  # Brightness variance (X-ray exposure differences)
    degrees=15,  # Welds can be at various angles
    translate=0.15,  # Weld position varies
    scale=0.3,  # Different weld sizes
    shear=5,  # Some shear is OK for industrial
    perspective=0.0002,  # Slight perspective from X-ray angle
    flipud=0.5,  # Defects have no orientation preference
    fliplr=0.5,  # Full horizontal flip
    mosaic=0.9,  # HIGH mosaic - great for small defects
    mixup=0.1,  # Some mixup helps with small defects
    copy_paste=0.0,  # Too slow, skip it
    auto_augment=None,
)

print(f"\nTraining complete!")
print(f"Best weights: {exp_dir / 'train/weights/best.pt'}")

# Save results
from google.colab import drive
drive.mount('/content/drive')
!cp -r {exp_dir} /content/drive/MyDrive/Datasets/Models/

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Training YOLOv8s for Weld X-ray Inspection
Target defects: porosity, cracks, lack of fusion, inclusions
Epochs: 50, Batch: 48
Expected time: ~7-8 hours
Expected mAP50: 0.65-0.70
Ultralytics 8.3.197 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=48, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Datasets/processed_balanced_final/processed_balanced_final/dataset.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic

KeyboardInterrupt: 